# tarfile test:

ClientError: ClientError: API error (404): manifest for 991775080983.dkr.ecr.us-east-1.amazonaws.com/sagemaker-processing-hmlr:deployment not found: manifest unknown: Requested image not found

In [1]:
import tarfile

In [10]:
tar = tarfile.open(f'src/model_deployment/docker.tar.gzip', "w:bz2")
for name in ["src/model_deployment"]:
    tar.add(name)
tar.close()

FileNotFoundError: [Errno 2] No such file or directory: 'src/model_deployment/docker.tar.gzip'

In [ ]:
push_image_ecr_deployment ('src/model_deployment/docker_ecr.sh',
                           'develope', 
                           's3://hmlr-dp-poc4-data/sagemaker/pipelines-05w16kg9zn6t-Train-Models-JyyIITRPsn/output/model.tar.gz')

# Push Docker Image to ECR:

In [23]:
import sys, os
os.path.abspath(os.path.join('..'))
from tools.utils import *

ModuleNotFoundError: No module named 'tools'

In [21]:
import sys, os
os.path.abspath(os.path.join('..'))

from tools.utils import *

'/home/ec2-user/SageMaker/hmlr-sow10-poc1-aws/src'

In [17]:
!pwd

/home/ec2-user/SageMaker/hmlr-sow10-poc1-aws/src/model_deployment


In [13]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'other_folder')))

from utils import *

In [8]:
from pathlib import Path
path = Path('(pwd)')
path

PosixPath('(pwd)')

In [9]:
import subprocess
from pathlib import Path

def push_image_ecr(ecr_prefix, model_artifact_path):
    """
    Function for getting initial data that is needed for the main function 
    
    Arguments::
        * ecr_prefix: string
            name for ecr repo
    Outputs:
         *     
    """
    docker_file_path = Path('/home/ec2-user/SageMaker/hmlr-sow10-poc1-aws/src/model_deployment/docker_ecr.sh')
    bashCommand = f"chmod +x {docker_file_path} {model_artifact_path}"
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    bashCommand = f"sh {docker_file_path} {ecr_prefix}"
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    
    return True

In [34]:
%%sh
cd container/src/models
ls


In [12]:
# branch_name = 'efactoring'
# push_image_ecr(branch_name)

In [13]:
# bashCommand = "chmod +x docker_ecr.sh"
# process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)

# bashCommand = "sh docker_ecr.sh seven"
# process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)

# output, error = process.communicate()
# error

# SageMaker Client Setup:

In [1]:
import boto3
from sagemaker import get_execution_role
from time import gmtime, strftime
import time

import json
import requests

In [3]:
client = boto3.client(service_name='sagemaker')
runtime__client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

#not used here
s3_bucket = 'hmlr-dp-poc1-data'

role = get_execution_role()

In [6]:
region = boto3.Session().region_name

# Model Creation

In [23]:
role

'arn:aws:iam::348583376301:role/service-role/AmazonSageMaker-ExecutionRole-20220216T084081'

In [3]:
def model_creation(s3_bucket, account_id, region, client, role ):
    """
    
    """
    model_name = 'poc1-ve1-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    model_url = 's3://{}/src/'.format(s3_bucket) 
    container = f'{account_id}.dkr.ecr.{region}.amazonaws.com/sagemaker-hmlr-efactoring:latest'

    print('Model name: ' + model_name)
    print('Model data Url: ' + model_url)
    print('Container image: ' + container)

    container = {
        'Image': container
    }

    create_model_response = client.create_model(
        ModelName = model_name,
        ExecutionRoleArn = role,
        Containers = [container])

    print("Model Arn: " + create_model_response['ModelArn'])
    return model_name
model_name = model_creation (s3_bucket, account_id, region, client )

Model name: poc1-ve1-2022-03-09-15-16-40
Model data Url: s3://hmlr-dp-poc1-data/src/
Container image: 348583376301.dkr.ecr.eu-west-2.amazonaws.com/sagemaker-hmlr-efactoring:latest
Model Arn: arn:aws:sagemaker:eu-west-2:348583376301:model/poc1-ve1-2022-03-09-15-16-40


# Endpoint Config Creation


In [4]:
def endpoint_config_creation(model_name ):
    """
    """
    endpoint_config_name = 'poc-ver-one-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    print(f'Endpoint config name: {endpoint_config_name} ')

    instance_type = 'ml.t2.medium'

    create_endpoint_config_response = client.create_endpoint_config(
        EndpointConfigName = endpoint_config_name,
        ProductionVariants=[{
            'InstanceType': instance_type,
            'InitialInstanceCount': 1,
            'InitialVariantWeight': 1,
            'ModelName': model_name,
            'VariantName': 'AllTraffic'}])

    end_point_arn = create_endpoint_config_response['EndpointConfigArn']
    print(f"Endpoint config Arn: {end_point_arn} ")
    return end_point_arn, endpoint_config_name
end_point_arn, endpoint_config_name = endpoint_config_creation(model_name)

Endpoint config name: poc-ver-one-config2022-03-09-15-16-42 
Endpoint config Arn: arn:aws:sagemaker:eu-west-2:348583376301:endpoint-config/poc-ver-one-config2022-03-09-15-16-42 


# Endpoint Creation

In [25]:
# endpoint_name = 'poc-one-endpoint-dev'
# client.delete_endpoint(EndpointName=endpoint_name)
# client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Endpoint Invocation

In [2]:
runtime__client = boto3.client(service_name='sagemaker-runtime')
endpoint_name = f'develop'

content_type = "application/json"
request_body = {"text_input": "my name is Sohrab and I'm sending this from Amazon AWS Sagemaker test",
               "pk": 81000}

#Serialize data for endpoint
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

#Endpoint invocation
response = runtime__client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload)

#Parse results
result = json.loads(response['Body'].read().decode())
result

{'label_number': '3',
 'label_name': 'Notices',
 'predict_proba': '[0.14916471 0.05118615 0.05711905 0.36729705 0.0748373  0.30039574]'}

In [39]:
!ls

0_1_srs_20220304_driver.ipynb  container      __init__.py
api_deployment.py	       docker_ecr.sh  __pycache__
